# Reduced-Complexity Particle Simulation in Python

<figure>
<img src="https://github.com/passaH2O/dorado/blob/master/docs/source/examples/images/logo.gif?raw=true" width="750"/>
<figcaption> Example of particles moving along a steady flow field </figcaption>
</figure>

<br>

### By the end of this lab you will be able to ...

- Simulate particle transport using the reduced-complexity model `dorado` like in the above animation

- Route particles across your Onion Creek domain from Lab 4, a dry streambed from Roy Guerrero Park, and some simulated delta landscapes

<br>

### Questions to think about:

- Does it matter _where_ particles are simulated? What might different locations signify within the context of a single river or delta?

- How do our reduced-complexity "rules" and associated parameters influence the movement of particles?

- How might you design a trip to a river or delta to validate or invalidate the results of your particle modeling?

- Is programming cool or fun?



Let's go ahead and download and uncompress all of the data we're going to use in this notebook. (Note: this might take a few minutes)

In [ ]:
%%capture

# onion creek and a channel at roy guerrero park
!wget 'https://utexas.box.com/shared/static/3y3fjz39qgsir73yxpaw0rkz29dhgcmw.pkl' -O 'OnionCreek.pkl'
!wget 'https://utexas.box.com/shared/static/7qr4hqneb3b30xc5amr4kv3vyjgp0sy2.pkl' -O 'RoyGPark.pkl'

# some pyDeltaRCM model outputs with different input sand fractions
!wget 'https://utexas.box.com/shared/static/tjvr9o7oyt6hthherhaubxlkx6d6e4m7.xz' -O 'pyDeltaRCM_outputs.tar.xz'
!tar -xf pyDeltaRCM_outputs.tar.xz

# Install _dorado_

Now we will install and import and Python packages we need to load the downloaded data, simulate our passive particles, and then visualize them.

In [ ]:
%%capture
!pip install pydorado  # install the particle routing package

In [ ]:
# libraries for loading our data
import pickle  # to access 'pickled' compressed data
import xarray as xr  # to load 'dimensional' data stored in netCDF files

# standard libraries for arrays and plots
import numpy as np  # base array manipulation library
import matplotlib.pyplot as plt  # standard plotting library

# library for particle simulation
import dorado

# Simulating Particles on Onion Creek

We are going to visualize gridded flow information from the final timestep of the ANUGA simulation you ran of Onion Creek (Lab 4).

In [ ]:
# this cell loads the data we downloaded
with open('OnionCreek.pkl', 'rb') as f:
  onioncreek = pickle.load(f)

In [ ]:
# see what variables are available for this dataset
onioncreek.keys()

In [ ]:
# now we will make the plot
fig, ax = plt.subplots(2, 2, figsize=(15, 10))  # sets up a 2x2 set of axes

ax[0, 0].imshow(onioncreek['stage'])  # shows the stage grid
ax[0, 0].set_title('stage')  # title for stage grid

ax[0, 1].imshow(onioncreek['depth'])  # shows the depth grid
ax[0, 1].set_title('depth')  # title for depth grid

ax[1, 0].imshow(onioncreek['qx'])  # plots variable qx
ax[1, 0].set_title('x-component of discharge')  # title

ax[1, 1].imshow(onioncreek['qy'])  # plots variable qy
ax[1, 1].set_title('y-component of discharge')  # title

plt.show()  # displays the plot

We can see the effect of this rough interpolation to a 5 m grid. For the purposes of demonstration, we will now drop some particles on the domain and let them travel down Onion Creek.

In [ ]:
# print resolution of the gridded fields
print(onioncreek['dx'])

Before we drop some particles onto the landscape, we need to figure out _where_ to put them. Our particles are designed to move through water, so we need to locate a channel.

In [ ]:
# plot the water depth and a line that we will investigate
plt.figure(figsize=(10, 10))  # draw a bigger figure
plt.imshow(onioncreek['depth'])  # plot depth
plt.colorbar(shrink=0.4)  # add colorbar
plt.plot(np.ones((50,))*150, np.linspace(125, 175), c='r')  # red line to look at in more detail
plt.show()

In [ ]:
# look at the water depths along that line
plt.plot(onioncreek['depth'][125:175, 150])
plt.show()

Now that we have a rough idea of where a channel is, we will set up our particles. To do this, we have to first define the hydrodynamic grid information such as the grid resolution, water depths, etc.

In [ ]:
# for any particle simulation, we start by defining model parameters
params = dorado.particle_track.modelParams()

# set the grid resolution
params.dx = onioncreek['dx']

# provide information about the flow field
params.depth = onioncreek['depth']
params.stage = onioncreek['stage'].copy()
params.qx = onioncreek['qx']
params.qy = onioncreek['qy']

# finally we tell the model what hydrodynamic model we are using
params.model = 'ANUGA'

Now that the model is configured, we can generate some particles. We need to decide _where_ to put the particles, _how many_ particles we want to seed, and then we can create them.

In [ ]:
# next we generate the particles themselves
particles = dorado.particle_track.Particles(params)  # supply model parameters

# decide where to place particles and how many
seed_xloc = np.arange(125, 175)
seed_yloc = [150]
Np_tracer = 50  # 50 particles

# generate particles
particles.generate_particles(Np_tracer, seed_xloc, seed_yloc)

So we've generated some particles! It is time to route them through the domain. To do this, we can define either a set number of _steps_ along the grid for them to take, or we can let them move until they reach a specified _time_.

We will be specifying a target time for the particles, we provide this time in units of _seconds_.

In [ ]:
# now route the particles for a set time (in seconds)
tt = 10*60*60  # 10 hours in seconds

walk_data = particles.run_iteration(target_time=tt)

Running the particle routing provided us with a Python dictionary `walk_data`.

`walk_data` contains information about the locations (in x-y space) and travel times, associated with each particle. While we can query this dictionary directly to make plots or access the data, there are a number of functions that will help us do this more easily. These "routines" are documented [here](https://passah2o.github.io/dorado/apiref/routines.html), below we will use the `plot_state` function to visualize the particles at different points in time.

In [ ]:
# plotting initial positions
dorado.routines.plot_state(onioncreek['stage'], walk_data, iteration=0, c='w')

In [ ]:
# plot every 15 minutes
from IPython.display import clear_output

fifteen = 15*60
for i in range(0, tt+fifteen, fifteen):
  clear_output(wait=True)
  dorado.routines.plot_state(
      onioncreek['stage'], walk_data, target_time=i, c='r')
  plt.title(str(i/60/60) + ' hours')
  plt.show()

# Simulating Particles in Roy Guerrero Park

Now we can take a look at the Roy Guerrero Park data. An ANUGA model similar to what you ran for Onion Creek was used here to create some hydrodynamic flow fields to drop particles onto. 

<figure>
<img src="https://utexas.box.com/shared/static/ucp1h3lxgy3e5yvhqld2l5xa700i22kc.png?raw=true" width="1000"/>
<figcaption> Google Maps Image of the park and our site </figcaption>
</figure>

We have to load the downloaded data into the Python notebook.

In [ ]:
with open('RoyGPark.pkl', 'rb') as f:
  royg = pickle.load(f)

Again we can query the available variables, and then we can visualize them.

In [ ]:
royg.keys()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

ax[0, 0].imshow(royg['stage'])
ax[0, 0].set_title('stage')

ax[0, 1].imshow(royg['depth'])
ax[0, 1].set_title('depth')

ax[1, 0].imshow(royg['qx'])
ax[1, 0].set_title('x-component of discharge')

ax[1, 1].imshow(royg['qy'])
ax[1, 1].set_title('y-component of discharge')

plt.show()

For all particle simulations, we begin by defining model parameters. This involves information about the hydrodynamic gridded fields, as well as parameters associated with the model's weighted random walk. Documentation on how to define the parameters is available [here](https://passah2o.github.io/dorado/background/index.html), and background on the random walk is also [documented](https://passah2o.github.io/dorado/userguide/index.html#defining-the-params).

In [ ]:
# for any particle simulation, we start by defining model parameters
params = dorado.particle_track.modelParams()

# set the grid resolution
params.dx = royg['dx']

# provide information about the flow field
params.depth = royg['depth']
params.stage = royg['stage'].copy()
params.qx = royg['qx']
params.qy = royg['qy']

# finally we tell the model what hydrodynamic model we are using
params.model = 'ANUGA'

There are a number of model parameters we can alter to fundamentally _change_ the way the particles move along the flow field. In this case, we will explicitly define them below, so that you can modify these parameters more easily if you wish. We start with default values.

In [ ]:
# the theta parameter - controls the dependence of random walk on water depth
params.theta = 1  # 1 = default, 0 is less dependence on depth, 2 is more

In [ ]:
# the gamma parameter - controls balance of water surface slope and inertial force on particle transport
params.gamma = 0.05  # default value - 5% based on surface slope, 95% based on discharge

In [ ]:
# "dry" depth - the minimum depth of water a cell must have for particles to consider it "wet"
params.dry_depth = 0.1  # default is 0.1 m

In [ ]:
# diff_coeff - diffusion coefficient for random walk travel times
# uncertaintly in travel time due to fixed grid means we want to add some uncertainty
params.diff_coeff = 0.2  # default value +/- 10% advective time estimate

In [ ]:
# steepest_descent - removes randomness from walk and particles transport to highest weighted cell
params.steepest_descent = False  # default is False, turn on by switching to True

We need to identify a potential seeding location to inject the particles.

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(royg['depth'])
plt.colorbar(shrink=0.6)
plt.plot(np.ones((50,))*50, np.linspace(100, 300), c='r')
plt.show()

In [ ]:
# plot the depth along the red line
plt.plot(royg['depth'][100:300, 50])
plt.show()

Now that we have an idea of where the channel is, we can seed some particles in that location. In this example we are dropping the particles in a single x-y location, but recall that you can provide a list of x and/or y indices to seed particles across a wider area if you would like.

In [ ]:
# next we generate the particles themselves
particles = dorado.particle_track.Particles(params)  # supply model parameters

# decide where to place particles and how many
seed_xloc = [175]
seed_yloc = [50]
Np_tracer = 10  # 10 particles

# generate particles
particles.generate_particles(Np_tracer, seed_xloc, seed_yloc)

In [ ]:
# now route the particles for a set time (in seconds)
tt = 5*60*60  # 5 hours in seconds

walk_data = particles.run_iteration(target_time=tt)

In [ ]:
# plotting initial positions
dorado.routines.plot_state(royg['depth'], walk_data, iteration=0, c='w')

In [ ]:
# plot every 15 minutes
from IPython.display import clear_output

fifteen = 15*60
for i in range(0, tt+fifteen, fifteen):
  clear_output(wait=True)
  dorado.routines.plot_state(
      royg['stage'], walk_data, target_time=i, c='r')
  plt.title(str(i/60/60) + ' hours')
  plt.show()

# Simulating Particles Over _pyDeltaRCM_ Model Deltas

Now we'll look at the same modeled deltas we inspected in the _pyDeltaRCM_ notebook, only this time we'll route passive particles through them.

First we'll load the 3 model outputs into a dictionary structure just like we did in the previous notebook.

In [ ]:
# access pyDeltaRCM outputs
data = dict()
for sf in [20, 50, 80]:
  # collect the model output
  data[sf] = xr.load_dataset('sandf/sf' + str(sf) + '/pyDeltaRCM_output.nc')

In [ ]:
data.keys()

In [ ]:
list(data[20].variables)

In [ ]:
data[20]['eta'].shape

Now you can define a sand fraction percentage, 20, 50, or 80, and run through the next few cells with that output. 

In [ ]:
# pick an output to visualize
# remember sf is the percentage of sand in the input sediment
sf = 20

We'll visualize the temporal evolution of the specific sand fraction output you chose in this next cell.

In [ ]:
# now loop through topography data to see model evolution
from IPython.display import clear_output

for i in range(data[sf]['eta'].shape[0]):
  # every 25 outputs
  if i % 25 == 0:
    clear_output(wait=True)
    fig, ax = plt.subplots()
    im = ax.imshow(data[sf].eta[i])
    plt.colorbar(im, ax=ax, shrink=0.6)
    ax.set_title('Sand Fraction: ' + str(sf) + '%, Output #: ' + str(i))
    plt.show()

Now we will drop particles onto this landscape. Note that unlike the previous examples, here we have a temporal record of the evolution of this landscape. So uniquely, we are able to choose an instant in time, over which to route our particles.

In the next cell you will define the _temporal index_ over which you'll then route some particles. An index value of 0, is the first saved model output, while an index of 1250 is the final output. 

In [ ]:
# select output index
# the output index is a measure of how much time has elapsed
# index 1,000 is later in the simulation than 500
output_index = 500

In [ ]:
# check the shape of the topography array, it is arranged as t-x-y (time, x, y)
data[sf]['eta'].shape

In [ ]:
# visualize delta variables at this time
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

ax[0, 0].imshow(data[sf]['eta'][output_index, :, :])
ax[0, 0].set_title('topography')

ax[0, 1].imshow(data[sf]['depth'][output_index, :, :])
ax[0, 1].set_title('depth')

ax[1, 0].imshow(data[sf]['velocity_x'][output_index, :, :])
ax[1, 0].set_title('x-component of velocity')

ax[1, 1].imshow(data[sf]['velocity_y'][output_index, :, :])
ax[1, 1].set_title('y-component of velocity')

plt.show()

In [ ]:
# define some parameters for the particles
params = dorado.particle_track.modelParams()

# set the grid resolution
params.dx = 50.0  # models were run at 50 m pixel resolution

# provide information about the flow field
params.depth = data[sf]['depth'][output_index, :, :].data
params.topography = data[sf]['eta'][output_index, :, :].data
params.u = data[sf]['velocity_x'][output_index, :, :].data
params.v = data[sf]['velocity_y'][output_index, :, :].data

# finally we tell the model what hydrodynamic model we are using
params.model = 'DeltaRCM'

In [ ]:
# next we generate the particles themselves
particles = dorado.particle_track.Particles(params)  # supply model parameters

# decide where to place particles and how many
seed_xloc = [10]
seed_yloc = np.arange(140, 170)
Np_tracer = 50  # 50 particles

# generate particles
particles.generate_particles(Np_tracer, seed_xloc, seed_yloc)

In [ ]:
# now route the particles for a set time (in seconds)
tt = 4*60*60  # 4 hours in seconds

walk_data = particles.run_iteration(target_time=tt)

In [ ]:
# plotting initial positions
dorado.routines.plot_state(data[sf]['eta'][output_index, :, :].data, walk_data, iteration=0, c='w')

In [ ]:
# plot every 15 minutes
from IPython.display import clear_output

fifteen = 15*60
for i in range(0, tt+fifteen, fifteen):
  clear_output(wait=True)
  dorado.routines.plot_state(
      data[sf]['eta'][output_index, :, :].data,
      walk_data, target_time=i, c='r')
  plt.title(str(i/60/60) + ' hours')
  plt.show()

Does it make sense to seed the particles in a line like that?

In [ ]:
# make a plot of the water depths and zoom in to see
dorado.routines.plot_state(data[sf]['depth'][output_index, :, :].data, walk_data, iteration=0, c='w')
plt.ylim([0, 20])
plt.xlim([135, 175])
plt.colorbar(shrink=0.6)
plt.show()

Maybe not - we drop some particles on cells that have water depths of 0.0 m - they are dry!

Can you devise a better particle seeding strategy?

# Some ideas / things you could try out

How do the model parameters (like `theta`, `gamma`, and `steepest_descent`) influence the behavior of the particles? What should we keep in mind when using any reduced-complexity model? 

<br>

Try changing the variable `sf` to simulate particles on top of different topographies, you've loaded model results with sand fractions of 20, 50, and 80%. How do you think the input sand fraction impacts the shape of the landscape or the movement of particles?

<br>

Or consider changing the `output_index` variable to examine the delta landscapes at different times in their evolution. How does the delta evolve over time? Does particle movement change as the landscape becomes more mature?

<br>

You can also change the location (`seed_xloc`, `seed_yloc`), the number of particles (`Np_tracer`) and the duration for which they are routed (`tt`). Does it matter where particles originate?

<br>

What can we learn from modeling like this? Can these kinds of exercises help us design better sediment diversions and build coastal land more efficiently?